In [34]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [5]:
# Load the dataset
data = pd.read_csv('diabetes.csv')

In [6]:
# Visualize dataset
data

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,63,negative
764,2,122,70,27,0,36.8,27,negative
765,5,121,72,23,112,26.2,30,negative
766,1,126,60,0,0,30.1,47,positive


In [7]:
# Extract X , y
X = data.iloc[:,0:-1].values
y_string = data.iloc[:, -1].values

In [11]:
print(X.shape, y_string.shape)

(768, 7) (768,)


In [25]:
# Convert y to a float from a string
y_int = []
for entry in y_string:
    if entry =='positive':
        y_int.append(1)
    else:
        y_int.append(0)
y = np.array(y_int, dtype='float64')
print(y.shape)

(768,)


In [26]:
# Data normalization
sc = StandardScaler()
X = sc.fit_transform(X)
X.shape

(768, 7)

In [27]:
# Convert to PyTorch tensors
x = torch.tensor(X)
y = torch.tensor(y).unsqueeze(1)

In [28]:
print(x.shape, y.shape)

torch.Size([768, 7]) torch.Size([768, 1])


In [29]:
class Dataset(Dataset):
    def __init__(self, x,y):
        self.x = x
        self.y = y
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [30]:
dataset = Dataset(x,y)

In [32]:
len(dataset), dataset[0]

(768,
 (tensor([ 0.6399,  0.8483,  0.1496,  0.9073, -0.6929,  0.2040,  1.4260],
         dtype=torch.float64), tensor([1.], dtype=torch.float64)))

In [36]:
# Load the data to your dataloader for batch preprocessing and shuffling
train_loader = torch.utils.data.DataLoader(dataset = dataset, batch_size = 32, shuffle = True)

In [37]:
train_loader

In [56]:
# Create network class
class Model(nn.Module):
    def __init__(self, input_features, output_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4,3)
        self.fc4 = nn.Linear(3,output_features)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out
        

In [57]:
# Create model
net = Model(input_features = 7, output_features = 1)
criterion = torch.nn.BCELoss(size_average = True)
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1, momentum = 0.9)



In [ ]:
epochs = 200
for epoch in range(epochs):
    for inputs, labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        # Forward prop
        outputs = net(inputs)
        # Loss calculation
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        #backprop
        loss.backward()
        # update weights
        optimizer.step()
    
    # Accuracy calculation
    output = (outputs >0.5).float()
    accuracy = (output == labels).float().mean()
    print(f"Epoch {epoch+1}/{}, Loss: {:.3f}, Accuracy: {:.3f}")